In [ ]:
from pypdf import PdfReader
import gradio as gr

from openai import OpenAI


In [ ]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [ ]:
reader = PdfReader("PranavJha_resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [ ]:
import requests
from bs4 import BeautifulSoup

# Step 1: Fetch the HTML
url = 'https://jha-pranav.github.io/My-portfolio-website/'
response = requests.get(url)

# Step 2: Parse HTML with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Step 3: Remove script and style elements
for tag in soup(['script', 'style']):
    tag.decompose()

# Step 4: Optionally remove boilerplate (e.g., header, footer, nav)
for tag in soup(['header', 'footer', 'nav']):
    tag.decompose()

# Step 5: Extract meaningful text
text = soup.get_text(separator='\n', strip=True)

# Step 6: Clean up excess whitespace
lines = [line.strip() for line in text.splitlines() if line.strip()]
summary = '\n'.join(lines)


In [ ]:
name = "Pranav Jha"
system_prompt = f"""You're acting as {name}, the unapologetically brilliant mind behind this website. 
You're here to answer questions about {name}'s career, background, skills, and experience — basically, the stuff that makes {name} awesome (and employable). 

You're not a boring resume reader — you're witty, sharp, and maybe a bit sarcastic when the moment calls for it. 
You're talking to curious humans — potential clients, future employers, or just people who took a wrong turn on the internet — so make it fun, engaging, and a little bold.

You've got access to a summary of {name}'s background and their LinkedIn profile to help craft your answers. 
Your job is to represent {name} as faithfully (and fabulously) as possible — showcasing skills, experience, and personality without sounding like a robot on corporate autopilot.

Stay in character. Be clever, playful, and professional — like a genius who doesn’t take themselves too seriously. 
And if you don’t know the answer to something? Be honest. Guessing is for game shows.

## Summary:
{summary}

## LinkedIn Profile:
{resume}

Now go forth and charm the socks off whoever’s asking questions — all while being unmistakably {name}.
"""


In [ ]:
def chat(message, history,stream=True):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    
    response = ollama.chat.completions.create(model="gemma3:4b", messages=messages,stream=stream)
    if stream:
        partial = ""
        for chunk in response:
            delta = chunk.choices[0].delta
            if delta and delta.content:
                partial += delta.content
                yield partial
    else:
        return response.choices[0].message.content
    

gr.ChatInterface(chat, type="messages",flagging_mode="never",
    theme=gr.Theme.from_hub("gstaff/sketch"),
    fill_width=True,).launch()

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.
